In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.api import add_constant, OLS
from matplotlib import pyplot as plt

In [2]:
cluster_data = pd.read_csv("../data/cluster.csv", index_col=0)
cluster_data.head()

,0,1,2,optics_label,kmeans_label,dbscan_label,hierarchy_label
A,13.799941,10.532969,-1.030204,0,1,0,4
AAL,-9.694422,-9.272758,0.516964,6,14,1,1
AAPL,0.214237,18.759165,-2.062645,4,12,2,9
ABBV,17.133244,3.435651,-7.875369,2,1,0,4
ABT,12.487381,8.524612,-5.642640,-1,1,0,4


In [3]:
raw_data = pd.read_csv("../data/raw_data.csv", index_col=0)
raw_data.head()

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,40.689999,40.910000,26.337500,57.610001,42.930000,22.950001,101.830002,91.970001,54.439999,35.840000,...,124.980133,29.83,68.769997,35.700001,77.459999,36.080002,51.912292,98.844658,66.489998,47.270000
2016-01-05,40.549999,40.520000,25.677500,57.369999,42.919998,23.033333,102.360001,92.339996,54.040001,36.240002,...,125.839996,29.90,70.070000,36.060001,78.120003,36.070000,51.782890,100.902916,64.820000,48.009998
2016-01-06,40.730000,41.230000,25.174999,57.380001,42.560001,23.070000,102.160004,91.019997,51.740002,35.360001,...,125.839996,29.24,66.440002,36.439999,77.470001,35.619999,51.416248,101.339806,62.230000,48.020000
2016-01-07,39.000000,40.450001,24.112499,57.209999,41.540001,23.046667,99.160004,89.110001,50.419998,34.520000,...,114.949997,28.16,60.189999,36.580002,76.230003,34.700001,49.662113,99.009712,59.410000,46.560001
2016-01-08,38.590000,40.369999,24.240000,55.650002,40.669998,22.806667,98.199997,87.849998,49.980000,34.389999,...,116.620003,27.90,57.740002,36.180000,74.690002,34.369999,48.986340,98.592232,59.250000,45.880001


In [4]:
def examine_cluster(stock_cluster, index, output_path):
    for i in range(len(index)):
        for j in range(i+1, len(index)):
            score, pvalue, _ = coint(stock_cluster[index[i]], stock_cluster[index[j]])
            if pvalue < 0.01:
                spread = compute_spread(stock_cluster, index[i], index[j])
                adf_pvalue = ADF_test(spread)
                if adf_pvalue < 0.01:
                    print(f"{index[i]} and {index[j]} are likely co-integrated.")
                    print(f"Coint pvalue = {pvalue:.6f} ADF test pvalue = {adf_pvalue:.6f}")
                    with open(output_path, 'a') as file:
                        file.write(f"{index[i]} and {index[j]} are likely co-integrated. ")
                        file.write(f"Coint pvalue = {pvalue:.6f} ADF test pvalue = {adf_pvalue:.6f}\n")

In [5]:
def compute_spread(data, stock1, stock2):
    # lp = np.log(data)
    # S1 = lp[stock1]
    S1 = data[stock1]
    S1 = add_constant(S1)
    # S2 = lp[stock2]
    S2 = data[stock2]
    linear_regression = OLS(S2, S1).fit()
    S1 = S1[stock1]
    beta = linear_regression.params[stock1]
    spread = S2 - beta * S1
    return spread

In [6]:
def ADF_test(spread):
    adf = adfuller(spread, maxlag=1)
    adf_pvalue = adf[1]
    return adf_pvalue

In [7]:
def identify_trading_pairs(raw_data, cluster_data, cluster_algorithm, cluster, output_path):
    # cluster_algorithm: optics, kmeans, dbscan, or hierarchy
    # cluster: 'all' or a specific cluster index
    open(output_path, 'w').close()
    cluster_algorithm += "_label"
    cluster_label = np.unique(cluster_data[cluster_algorithm])
    if cluster == 'all':
        for each in cluster_label:
            if each == -1: continue
            with open(output_path, 'a') as file:
                file.write(f"Cluster {each}:\n")
            print(f"Cluster {each}:")
            index = cluster_data[cluster_data[cluster_algorithm] == each].index
            stock_cluster = raw_data.loc[:, index]
            examine_cluster(stock_cluster, index, output_path)

    else:
        with open(output_path, 'a') as file:
            file.write(f"Cluster {cluster}:\n")
        print(f"Cluster {cluster}:")
        index = cluster_data[cluster_data[cluster_algorithm] == int(cluster)].index
        stock_cluster = raw_data.loc[:, index]
        examine_cluster(stock_cluster, index, output_path)

In [8]:
identify_trading_pairs(raw_data, cluster_data, "hierarchy", 'all', "./trading_pairs/trading_pairs_1.txt")

Cluster 0:
AME and WMT are likely co-integrated.
Coint pvalue = 0.003705 ADF test pvalue = 0.000651
AOS and NWS are likely co-integrated.
Coint pvalue = 0.006915 ADF test pvalue = 0.003330
EXPD and HD are likely co-integrated.
Coint pvalue = 0.001545 ADF test pvalue = 0.000293
FAST and NDSN are likely co-integrated.
Coint pvalue = 0.004490 ADF test pvalue = 0.000390
FAST and WMT are likely co-integrated.
Coint pvalue = 0.005454 ADF test pvalue = 0.001211
HD and NDSN are likely co-integrated.
Coint pvalue = 0.000663 ADF test pvalue = 0.000085
HD and UNP are likely co-integrated.
Coint pvalue = 0.004881 ADF test pvalue = 0.000357
HPE and RL are likely co-integrated.
Coint pvalue = 0.003530 ADF test pvalue = 0.009752
JBHT and LOW are likely co-integrated.
Coint pvalue = 0.002063 ADF test pvalue = 0.000223
LOW and NDSN are likely co-integrated.
Coint pvalue = 0.002583 ADF test pvalue = 0.000139
NDSN and WMT are likely co-integrated.
Coint pvalue = 0.004861 ADF test pvalue = 0.002134
Cluste

In [9]:
identify_trading_pairs(raw_data, cluster_data, "dbscan", 'all', "./trading_pairs/trading_pairs_2.txt")

Cluster 0:
A and DGX are likely co-integrated.
Coint pvalue = 0.003580 ADF test pvalue = 0.000161
ABT and ZTS are likely co-integrated.
Coint pvalue = 0.002207 ADF test pvalue = 0.000452
AMGN and VRTX are likely co-integrated.
Coint pvalue = 0.007111 ADF test pvalue = 0.001433
DGX and DHR are likely co-integrated.
Coint pvalue = 0.000676 ADF test pvalue = 0.007628
DGX and JNJ are likely co-integrated.
Coint pvalue = 0.005788 ADF test pvalue = 0.000384
DGX and MTD are likely co-integrated.
Coint pvalue = 0.000074 ADF test pvalue = 0.000269
DGX and RMD are likely co-integrated.
Coint pvalue = 0.006343 ADF test pvalue = 0.005138
DGX and TMO are likely co-integrated.
Coint pvalue = 0.000497 ADF test pvalue = 0.005356
DGX and WAT are likely co-integrated.
Coint pvalue = 0.007992 ADF test pvalue = 0.003916
EW and RMD are likely co-integrated.
Coint pvalue = 0.004184 ADF test pvalue = 0.001029
EW and TECH are likely co-integrated.
Coint pvalue = 0.006274 ADF test pvalue = 0.001685
HOLX and MT

In [10]:
identify_trading_pairs(raw_data, cluster_data, "optics", 'all', "./trading_pairs/trading_pairs_3.txt")

Cluster 0:
HOLX and MTD are likely co-integrated.
Coint pvalue = 0.002590 ADF test pvalue = 0.000694
HOLX and TMO are likely co-integrated.
Coint pvalue = 0.003647 ADF test pvalue = 0.002037
IQV and WAT are likely co-integrated.
Coint pvalue = 0.009555 ADF test pvalue = 0.003029
RVTY and TECH are likely co-integrated.
Coint pvalue = 0.007406 ADF test pvalue = 0.003950
Cluster 1:
ISRG and SYK are likely co-integrated.
Coint pvalue = 0.009846 ADF test pvalue = 0.001687
Cluster 2:
AMGN and VRTX are likely co-integrated.
Coint pvalue = 0.007111 ADF test pvalue = 0.001433
Cluster 3:
AON and FDS are likely co-integrated.
Coint pvalue = 0.002551 ADF test pvalue = 0.000579
ICE and MCO are likely co-integrated.
Coint pvalue = 0.004215 ADF test pvalue = 0.001041
ICE and ROP are likely co-integrated.
Coint pvalue = 0.008799 ADF test pvalue = 0.002011
ICE and SHW are likely co-integrated.
Coint pvalue = 0.000015 ADF test pvalue = 0.000028
ICE and WTW are likely co-integrated.
Coint pvalue = 0.0021